*TP réalisé par Rachel Blin dans le cadre du cours d'Alexandrina Rogozan*

# Codage de Shannon-Fano

L'objectif de ce TP est de réaliser le codage de Shannon-Fano pour le message suivant :  
  
"*Il n'existe que deux choses infinies, l'univers et la bêtise humaine... mais pour l'univers, je n'ai pas de certitude absolue.*" (Albert Einstein)

## Simplification de la séquence

Afin de rendre la séquence plus simple à encoder, nous allons tout d'abord transformer la phrase de telle sorte à ce qu'elle ne contienne que des caractères présents dans les 26 lettres de l'alphabet en minuscule, sans accents, ainsi que les espaces.

La message à encoder devient donc :  
  
"*il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue*"

## Classification des symboles de la séquence par ordre d'occurences croissants

La première étape du codage de Shannon-Fano est de classer les symboles de la séquence à encoder par nombre d'occurences croissants.  
  
1) Dans un premier temps, répertoriez tous les caractères présents dans le message.

In [3]:
m = "il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue"

def unique(message):
    """ Helper function that return a list of the unique characters in the input message.
    
    # Argument:
        - message: The input string to be processed.
    
    # Return:
        A list containing all the unique characters in the input string.
    """
    chars = [] 
    for char in message: 
        if char not in chars: 
            chars.append(char) 
    return chars

chars = unique(m)
print(chars)

['i', 'l', ' ', 'n', 'e', 'x', 's', 't', 'q', 'u', 'd', 'c', 'h', 'o', 'f', 'v', 'r', 'a', 'b', 'm', 'p', 'j']


2) Calculez maintenant le nombre d'occurences de chacun de ces caractères dans le message et rangez ces occurences par nombre croissant.

In [7]:
import operator

def occurencies(message):
    """ Function that counts the number of occurences in a message and sort the occurencies by increasing order.
    
    # Argument:
        - message: The input string to be processed.
    
    # Return:
        A list of tuples containing each character and its number of occurences in the message.
    """
    occurences = []
    chars = unique(message)
    for char in chars:
        nb = message.count(char)
        occurences.append((char, nb))
        occ2 = occurences.copy()
        occ2.sort(key=lambda x: x[1]) # deuxième façon de faire un tri
    return sorted(occurences, key=lambda x: x[1]), occ2

sorted_occurences, sort_occurences = occurencies(m)
print(sorted_occurences)
print(sort_occurences)

[('q', 1), ('f', 1), ('j', 1), ('x', 2), ('c', 2), ('h', 2), ('v', 2), ('b', 2), ('m', 2), ('p', 2), ('d', 3), ('o', 3), ('r', 4), ('l', 5), ('t', 5), ('a', 6), ('n', 7), ('u', 8), ('s', 10), ('i', 12), ('e', 17), (' ', 19)]
[('q', 1), ('f', 1), ('j', 1), ('x', 2), ('c', 2), ('h', 2), ('v', 2), ('b', 2), ('m', 2), ('p', 2), ('d', 3), ('o', 3), ('r', 4), ('l', 5), ('t', 5), ('a', 6), ('n', 7), ('u', 8), ('s', 10), ('i', 12), ('e', 17), (' ', 19)]


## Séparation des symboles en deux sous-groupes

Une fois les symboles classés par ordre d'occurence croissante, il est nécessaire de créer un graphe permettant de séparer les symboles en deux sous-groupes de façon à ce que le nombre total d'occurences soit sensiblement égal entre les deux sous-groupes. Il faut répéter l'opération de telle sorte à ce qu'à la fin du graphe, il ne reste plus q'un seul caractère dans chacune des branches. Ensuite, il faut attribuer au sous-groupe de gauche la valeur 0 et la valeur 1 au sous-groupe de droite.

__Exemple__ :  
Pour une séquence contenant les caractères (a, b, c, d, e) d'occurences respectives (1, 1, 2, 2, 3) on obtient le graphe suivant : 

![graphe](TP_Shannon_Fano.png)

Ce qui donne donc le codage de Shannon-Fano suivant : 

| Caractère | Nombre d'occurences | Code Shannon-Fano |
|-----------|---------------------|-------------------|
| a         | 1                   | 000               |
| b         | 1                   | 001               |
| c         | 2                   | 01                |
| d         | 2                   | 10                |
| e         | 3                   | 11                |

3) Créez le graphe de séparation des symboles.

In [3]:
# Tree structure to hold the encoding data
class Tree(object):
    def __init__(self):
        self.g = None
        self.d = None
        self.data = None

def split_occurencies(sorted_occurencies):
    """ Split the sorted occurencies in half by the number of occurencies
    
    # Argument:
        - sorted_occurencies: List of tuple of the char sorted by the number of occurencies

    # Return:
        - Two lists of occurencies
    """
    if len(sorted_occurencies) == 2:
        return sorted_occurencies[:1], sorted_occurencies[1:]
    total_occurencies = sum([count for _, count in sorted_occurencies])
    middle = total_occurencies // 2
    
    cum_sum = 0
    counter = 0
    
    while cum_sum < middle:
        cum_sum += sorted_occurencies[counter][1]
        counter += 1
    
    return sorted_occurencies[:counter], sorted_occurencies[counter:]


def Shannon_Fano(occurencies):
    """ Recursive function that encodes the coding tree for shannon fano coding.
    
    # Argument:
        - occurencies: The list of tuples containing the sorted occurencies

    # Return:
        - The coding tree corresponding the the given list of occurencies.
    """
    # Sets the data of the current node
    node = Tree()
    node.data = "".join([value[0] for value in occurencies])
    #node.data = occurencies
    
    # Separate the list of occurencies into two subgroups of approximately the same number of occurencies
    # If the list cannot be separated, we stop the recursion here
    if len(occurencies) == 1:
        return node
    
    left_occurencies, right_occurencies = split_occurencies(occurencies)
    
    # Encode the occurencies in the sub-trees
    node.g = Shannon_Fano(left_occurencies)
    node.d = Shannon_Fano(right_occurencies)

    return node

def display_graph(root_node):
    """ Helper function to display the coding graph
    
    # Argument:
        - root_node: The root node of the coding tree

    # Return:
        - None
    """
    current_level = [root_node]
    nb_nodes = ""
    level = 0
    while current_level:
        next_level = list()
        nb_nodes = nb_nodes[:len(nb_nodes)//2]
        for node in current_level:
            print("level : ", level)
            print(nb_nodes + str(node.data))
            if node.g:
                next_level.append(node.g)
            if node.d:
                next_level.append(node.d)
            
            current_level = next_level
        level +=1
          
def get_dict_codage(node):
    """ Take a coding tree and return the associated coding dictionnary.
    
    # Argument:
        - node: The root node of the coding tree.

    # Return:
        - Dictionnary: The coding dictionnary.
    """
    dict_codage = {}
    
    coding_value = ""
    
    def recursive_call(coding_value, dict_codage, node):
        if len(node.data) == 1:
            dict_codage[node.data] = coding_value
        else:
            recursive_call(coding_value + "0", dict_codage, node.g)
            recursive_call(coding_value + "1", dict_codage, node.d)
    
    recursive_call(coding_value, dict_codage, node)
    
    return dict_codage
    
codage_Shannon_Fano = Shannon_Fano(sorted_occurences) # Graphe de séparation des symboles
dict_codage_Shannon_Fano = get_dict_codage(codage_Shannon_Fano)

print("Affichage du graphe de séparation des symboles :")
display_graph(codage_Shannon_Fano)
print("\n")
print("Affichage du dictionnaire des encodages de Shanon-Fano par caractère :")
print(dict_codage_Shannon_Fano)

Affichage du graphe de séparation des symboles :
level :  0
qfjxchvbmpdorltanusie 
level :  1
qfjxchvbmpdorltanu
level :  1
sie 
level :  2
qfjxchvbmpdorl
level :  2
tanu
level :  2
sie
level :  2
 
level :  3
qfjxchvbmp
level :  3
dorl
level :  3
tan
level :  3
u
level :  3
si
level :  3
e
level :  4
qfjxch
level :  4
vbmp
level :  4
dor
level :  4
l
level :  4
ta
level :  4
n
level :  4
s
level :  4
i
level :  5
qfjx
level :  5
ch
level :  5
vb
level :  5
mp
level :  5
do
level :  5
r
level :  5
t
level :  5
a
level :  6
qf
level :  6
jx
level :  6
c
level :  6
h
level :  6
v
level :  6
b
level :  6
m
level :  6
p
level :  6
d
level :  6
o
level :  7
q
level :  7
f
level :  7
j
level :  7
x


Affichage du dictionnaire des encodages de Shanon-Fano par caractère :
{'q': '0000000', 'f': '0000001', 'j': '0000010', 'x': '0000011', 'c': '000010', 'h': '000011', 'v': '000100', 'b': '000101', 'm': '000110', 'p': '000111', 'd': '001000', 'o': '001001', 'r': '00101', 'l': '0011', 't': '01000',

4) Effectuez maintenant le codage de Shannon-Fano.

In [5]:
def encode_message(m, dict_codage_Shannon_Fano):
    """ Helper function to encode a given message.
    
    # Arguments:
        - m: String, the message to encode
        - dict_coding_Shannon_Fano: Dictionnary containing the coding mapping

    # Return:
        - String, the coded message.
    """
    message_Shanon_Fano = ""
    for char in m:
        message_Shanon_Fano = message_Shanon_Fano + dict_codage_Shannon_Fano[char]
    return message_Shanon_Fano

message_Shanon_Fano = encode_message(m, dict_codage_Shannon_Fano)
print(message_Shanon_Fano)

10010011110101101000001110011000010001011100000000111011100100010101100000111100001000001100100110001011000111001010100000011001010110011011000110011011010110010001001010010110001110101000110011010011100010110101000100110001011100001101100011001001100101011011100011001001100110001100011100100101100101110011011010110010001001010010110001100000101011101010100110011100011101001100011001000101110000101010010101000100101000011001000101110100100010110000010010011011101


## Visualisation de l'encodage 

En se basant sur le codage binaire des caractères de la table ASCII, le message de départ est le suivant :  
"0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"

5) Que peut-on dire du rôle du codage de Shannon-Fano sur le message en entrée?

In [7]:
binaire = "0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"

print("longueur du message en binaire : ", len(binaire))
print("longueur du message codage de Shannon-Fano : ", len(message_Shanon_Fano))

longueur du message en binaire :  928
longueur du message codage de Shannon-Fano :  467


On constate donc que la longueur du message a été divisée par deux environ après encodage de Shannon-Fano.